<a href="https://colab.research.google.com/github/philschmid/efsync/blob/main/examples/efsync_pip_packages_and_s3_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example Notebook for efsync installing pip packages and downloading files from s3

this notebooks includes examples for the `CLI` and the `SDK`. 

This Notebook can be runned in google colab. 

In [2]:
!pip3 install efsync

     |████████████████████████████████| 215kB 5.4MB/s 
     |████████████████████████████████| 962kB 8.1MB/s 
     |████████████████████████████████| 2.6MB 14.2MB/s 
     |████████████████████████████████| 71kB 6.5MB/s 


In [45]:
%cd ..
!python3 setup.py sdist bdist_wheel
! pip install dist/efsync-0.9.20-py3-none-any.whl 
%cd examples 

/Users/philippschmid/projects/personal/efsync
['efsync', 'efsync.test', 'efsync.utils', 'efsync.utils.config', 'efsync.utils.security_group', 'efsync.utils.ssh', 'efsync.utils.iam_profile', 'efsync.utils.efs', 'efsync.utils.ec2']
running sdist
running egg_info
writing efsync.egg-info/PKG-INFO
writing dependency_links to efsync.egg-info/dependency_links.txt
writing entry points to efsync.egg-info/entry_points.txt
writing requirements to efsync.egg-info/requires.txt
writing top-level names to efsync.egg-info/top_level.txt
reading manifest file 'efsync.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'efsync.egg-info/SOURCES.txt'
running check
creating efsync-0.9.20
creating efsync-0.9.20/efsync
creating efsync-0.9.20/efsync.egg-info
creating efsync-0.9.20/efsync/test
creating efsync-0.9.20/efsync/utils
creating efsync-0.9.20/efsync/utils/config
creating efsync-0.9.20/efsync/utils/ec2
creating efsync-0.9.20/efsync/utils/efs
creating efsync-0.9.20/efsync/

adding 'efsync/utils/config/load_args_from_yaml.py'
adding 'efsync/utils/config/load_config.py'
adding 'efsync/utils/config/read_requirements_from_file.py'
adding 'efsync/utils/ec2/__init__.py'
adding 'efsync/utils/ec2/create_user_data.py'
adding 'efsync/utils/ec2/custom_waiter.py'
adding 'efsync/utils/ec2/ec2_main.py'
adding 'efsync/utils/ec2/ec2_waiter.py'
adding 'efsync/utils/efs/__init__.py'
adding 'efsync/utils/efs/describe_efs.py'
adding 'efsync/utils/iam_profile/__init__.py'
adding 'efsync/utils/iam_profile/iam_profile.py'
adding 'efsync/utils/security_group/__init__.py'
adding 'efsync/utils/security_group/ec2_security_group.py'
adding 'efsync/utils/ssh/__init__.py'
adding 'efsync/utils/ssh/createSSHClient.py'
adding 'efsync/utils/ssh/scp_to_ec2.py'
adding 'efsync/utils/ssh/ssh_key.py'
adding 'efsync-0.9.20.dist-info/LICENSE'
adding 'efsync-0.9.20.dist-info/METADATA'
adding 'efsync-0.9.20.dist-info/WHEEL'
adding 'efsync-0.9.20.dist-info/entry_points.txt'
adding 'efsync-0.9.20.di

In [44]:
!pip uninstall efsync -y

Found existing installation: efsync 0.9.19
Uninstalling efsync-0.9.19:
  Successfully uninstalled efsync-0.9.19


# Check installed version

In [46]:
!pip freeze | grep efsync

efsync==0.9.20


In [16]:
!efsync -h

usage: efsync <command> [<args>]

optional arguments:
  -h, --help            show this help message and exit
  --version, -v         show program version
  --requirements REQUIREMENTS, -r REQUIREMENTS
                        path of your requirements.txt
  --config_file CONFIG_FILE, -cf CONFIG_FILE
                        path of your efsync.yaml
  --python_version PYTHON_VERSION, -py PYTHON_VERSION
                        Python version used to install dependencies
  --efs_pip_dir EFS_PIP_DIR, -epd EFS_PIP_DIR
                        directory where the pip packages will be installed on
                        efs
  --efs_filesystem_id EFS_FILESYSTEM_ID, -efi EFS_FILESYSTEM_ID
                        File System ID from the EFS filesystem
  --clean_efs CLEAN_EFS, -ce CLEAN_EFS
                        Defines if the efs should be cleaned up before
                        uploading
  --file_dir FILE_DIR, -fd FILE_DIR
                        directory where all other files will be place

In [17]:
from efsync import efsync

---

## Create `requirements.txt` 

In [7]:
%%writefile ./requirements.txt

pyjokes
pandas
numpy
requests

Overwriting ./requirements.txt


# Example with `efsync.yaml`

In [8]:
%%writefile ./efsync.yaml

#standard configuration
efs_filesystem_id: fs-2226b27a # aws efs filesystem id (moint point)
subnet_Id: subnet-17f97a7d # subnet of which the efs is running in
ec2_key_name: efsync-asd913fjgq3 # required key name for starting the ec2 instance
clean_efs: all # Defines if the EFS should be cleaned up before. values: `'all'`,`'pip'`,`'file'` uploading
# aws profile configuration
aws_profile: schueler-vz # aws iam profile with required permission configured in .aws/credentials
aws_region: eu-central-1 # the aws region where the efs is running

# pip packages configurations
efs_pip_dir: lib # pip directory on ec2
python_version: 3.8 # python version used for installing pip packages -> should be used as lambda runtime afterwads
requirements: requirements.txt # path + file to requirements.txt which holds the installable pip packages

Overwriting ./efsync.yaml


In [9]:
from efsync import efsync

efsync('./efsync.yaml')

2020-10-25 17:55:55,061 - efsync - starting....
2020-10-25 17:55:55,062 - efsync - loading config
2020-10-25 17:55:55,077 - efsync - creating security group
2020-10-25 17:55:55,567 - efsync - loading default security group
2020-10-25 17:55:55,716 - efsync - creating ssh key for scp in memory
2020-10-25 17:55:56,335 - efsync - starting ec2 instance with security group sg-0f25eeb79a9ce2478 and subnet_Id subnet-17f97a7d
2020-10-25 18:00:59,841 - efsync - stopping ec2 instance with instance id i-034ae8dbb7b99f9e5
2020-10-25 18:01:45,491 - efsync - deleting iam profile
2020-10-25 18:01:46,528 - efsync - deleting ssh key
2020-10-25 18:01:46,712 - efsync - deleting security group
2020-10-25 18:01:47,022 - efsync - #################### finished after 5.866 minutes ####################


True

# Example with pyton `dict`

In [10]:
config = {
  'efs_filesystem_id': 'fs-2226b27a', # aws efs filesystem id (moint point)
  'subnet_Id': 'subnet-17f97a7d', # subnet of which the efs is running in
  'ec2_key_name':'efsync-asd913fjgq3',  # required key name for starting the ec2 instance
  'clean_efs': 'all', # Defines if the EFS should be cleaned up before. values: `'all'`,`'pip'`,`'file'` uploading
  'aws_profile': 'schueler-vz', # aws iam profile with required permission configured in .aws/credentials
  'aws_region': 'eu-central-1', # the aws region where the efs is running
  'efs_pip_dir': 'lib',  # pip directory on ec2
  'python_version': 3.8,  # python version used for installing pip packages -> should be used as lambda runtime afterwads
  'requirements': 'requirements.txt', # path + file to requirements.txt which holds the installable pip packages
}

In [18]:
from efsync import efsync


efsync(config)

2020-10-25 18:09:42,130 - efsync - starting....
2020-10-25 18:09:42,131 - efsync - loading config
2020-10-25 18:09:42,143 - efsync - creating security group
2020-10-25 18:09:42,523 - efsync - loading default security group
2020-10-25 18:09:42,687 - efsync - creating ssh key for scp in memory
2020-10-25 18:09:43,214 - efsync - starting ec2 instance with security group sg-0baacfbb38b8c541d and subnet_Id subnet-17f97a7d
2020-10-25 18:15:01,903 - efsync - stopping ec2 instance with instance id i-023f3dfead9e2fd2f
2020-10-25 18:15:47,713 - efsync - deleting iam profile
2020-10-25 18:15:49,364 - efsync - deleting ssh key
2020-10-25 18:15:49,559 - efsync - deleting security group
2020-10-25 18:15:49,859 - efsync - #################### finished after 6.128833333333334 minutes ####################


True

# Example with cli and `efsync.yaml`

In [23]:
!efsync -cf efsync.yaml

2020-10-25 18:19:47,236 - efsync - starting....
2020-10-25 18:19:47,236 - efsync - loading config
2020-10-25 18:19:47,251 - efsync - creating security group
2020-10-25 18:19:47,728 - efsync - loading default security group
2020-10-25 18:19:47,878 - efsync - creating ssh key for scp in memory
2020-10-25 18:19:48,432 - efsync - starting ec2 instance with security group sg-091caa95c9ae7130c and subnet_Id subnet-17f97a7d
{'Arn': 'arn:aws:iam::770710496542:instance-profile/efsync_instance_profile_role', 'Name': 'efsync_instance_profile_role'}
2020-10-25 18:24:52,266 - efsync - stopping ec2 instance with instance id i-0efa9c6b09f735de8
2020-10-25 18:25:37,931 - efsync - deleting iam profile
2020-10-25 18:25:39,057 - efsync - deleting ssh key
2020-10-25 18:25:39,251 - efsync - deleting security group
2020-10-25 18:25:39,551 - efsync - #################### finished after 5.871833333333333 minutes ####################


# Example with cli and parameters

In [47]:
!efsync --efs_filesystem_id  fs-2226b27a \
        --subnet_Id subnet-17f97a7d \
        --ec2_key_name efsync-asd913fjgq3 \
        --clean_efs all \
        --aws_profile schueler-vz \
        --aws_region eu-central-1 \
        --efs_pip_dir lib \
        --python_version 3.8 \
        --requirements requirements.txt

2020-10-25 18:40:00,587 - efsync - starting....
2020-10-25 18:40:00,587 - efsync - loading config
2020-10-25 18:40:00,597 - efsync - creating security group
2020-10-25 18:40:01,073 - efsync - loading default security group
2020-10-25 18:40:01,246 - efsync - creating ssh key for scp in memory
2020-10-25 18:40:01,818 - efsync - starting ec2 instance with security group sg-007f66499e888edab and subnet_Id subnet-17f97a7d
2020-10-25 18:45:15,757 - efsync - stopping ec2 instance with instance id i-0090f12b389b9c076
2020-10-25 18:46:01,407 - efsync - deleting iam profile
2020-10-25 18:46:02,651 - efsync - deleting ssh key
2020-10-25 18:46:02,862 - efsync - deleting security group
2020-10-25 18:46:03,158 - efsync - #################### finished after 6.042833333333333 minutes ####################
